# Introduction 

This example requires QCoDeS

In [1]:
from collections import defaultdict
import numpy as np 

from pyvisa_mock.base.base_mocker import BaseMocker, scpi
from pyvisa_mock.base.register import register_resource
from qcodes import VisaInstrument, InstrumentChannel
from qcodes.instrument_drivers.stahl import Stahl

The following is an example of a multi-channel voltage source. Notice that we can modularize the 
mocker, whereby each channel is handled by its own object. 

In [3]:
class MockerChannel(BaseMocker): 
    """
    A mocker channel for a multi channel voltage source. 
    Voltages are zero by default
    """
    
    def __init__(self): 
        self._voltage = 0
    
    # Lets define handler functions. Notice how we can be 
    # lazy in our regular expressions (using ".*"). The 
    # typehints will be used to cast strings to the 
    # required types
    
    @scpi(r":VOLT (.*)") #  E.g. handle ':INSTR:CHANNEL1:VOLT 2.3' 
    def _set_voltage(self, value: float) -> None:
        self._voltage = value
    
    @scpi(r":VOLT\?")
    def _get_voltage(self) -> float: 
        return self._voltage


class Mocker(BaseMocker):
    """
    The main mocker class. 
    """

    def __init__(self):
        self._channels = defaultdict(MockerChannel)

    @scpi("\*IDN\?")
    def idn(self) -> str: 
        """
        'vendor', 'model', 'serial', 'firmware'
        """
        return "Mocker,testing,00000,0.01"
    
    @scpi(r":INSTR:CHANNEL(.*)")
    def _get_channel(self, channel: int) -> MockerChannel:
        return self._channels[channel] 

Let's inspect which SCPI commands have been defined 

In [14]:
Mocker.__scpi_dict__

{'\\*IDN\\?': <visa_mock.base.base_mocker.SCPIHandler at 0x2c0d0ed61d0>,
 ':INSTR:CHANNEL(.*):VOLT (.*)': <visa_mock.base.base_mocker.SCPIHandler at 0x2c0d0ed6390>,
 ':INSTR:CHANNEL(.*):VOLT\\?': <visa_mock.base.base_mocker.SCPIHandler at 0x2c0d0ed63c8>}

In [4]:
class MyInstrumentChannel(InstrumentChannel): 
    def __init__(self, parent, name, channel): 
        super().__init__(parent, name)
        
        self.add_parameter(
            "voltage", 
            get_cmd=f":INSTR:CHANNEL{channel}:VOLT?", 
            set_cmd=f":INSTR:CHANNEL{channel}:VOLT {{}}", 
            get_parser=float, 
            unit="V"
        )
        
class MyInstrument(VisaInstrument): 
    
    n_channels = 10
    
    def __init__(self, name, address, **kwargs): 
        super().__init__(name, address, **kwargs)
        
        for channel in range(self.n_channels): 
            
            submodule = MyInstrumentChannel(
                self, 
                f"channel{channel}", 
                channel
            )
            
            self.add_submodule(
                f"channel{channel}", 
                submodule
            )
            
        self.connect_message()

In [5]:
register_resource("MOCK0::mock1::INSTR", Mocker())

In [6]:
my_instrument = MyInstrument("myinst", "MOCK0::mock1::INSTR", visalib="@mock")

Connected to: Mocker testing (serial:00000, firmware:0.01) in 0.03s


In [7]:
my_instrument.channel1.voltage()

0.0

In [8]:
my_instrument.channel1.voltage(2.1)

In [9]:
my_instrument.channel1.voltage()

2.1

In [10]:
my_instrument.channel2.voltage(-2.1)

In [11]:
my_instrument.channel2.voltage()

-2.1

In [12]:
my_instrument.channel1.voltage()

2.1